# Dataset

In [1]:
import math

import torch
import pandas as pd
import numpy as np

In [2]:
from pathlib import Path

DATA_PATH = Path('data/')
DATA_PATH.mkdir(parents=True, exist_ok=True)

DATA_CACHE = Path('data/cache_dir/')
DATA_CACHE.mkdir(parents=True, exist_ok=True)

DATA_PATH_SAVE_MODELS = Path('data/models/')
DATA_PATH_SAVE_MODELS.mkdir(parents=True, exist_ok=True)

DATA_SYNTHETIC = Path('synthetic/')
DATA_SYNTHETIC.mkdir(parents=True, exist_ok=True)

pd.set_option('display.max_colwidth', 500) 

## Facebook Data

In [3]:
facebook_df = pd.read_excel(DATA_PATH / 'facebook_data.xlsx', skiprows=1)
facebook_df.head(1)

,Group Name,User Name,Facebook Id,Page Category,Page Admin Top Country,Page Description,Page Created,Likes at Posting,Followers at Posting,Post Created,...,Message,Link,Final Link,Image Text,Link Text,Description,Sponsor Id,Sponsor Name,Sponsor Category,Overperforming Score (weighted — Likes 1x Shares 1x Comments 1x Love 1x Wow 1x Haha 1x Sad 1x Angry 1x Care 1x )
0,COVID19: Real Talk from Health Care Workers around the Globe,NaN,1073058046385811,none,NaN,NaN,NaN,66994.0,NaN,2020-04-01 22:42:32 EDT,...,We extubated 2 covid patients today and they are doing awesome! Should be on a tele floor tomorrow! (TX),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,189.33


In [4]:
facebook_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6860 entries, 0 to 6859
Data columns (total 40 columns):
 #   Column                                                                                                              Non-Null Count  Dtype  
---  ------                                                                                                              --------------  -----  
 0   Group Name                                                                                                          6860 non-null   object 
 1   User Name                                                                                                           0 non-null      float64
 2   Facebook Id                                                                                                         6860 non-null   int64  
 3   Page Category                                                                                                       6860 non-null   object 
 4   Page Admin Top Cou

In [5]:
facebook_df.columns

Index(['Group Name', 'User Name', 'Facebook Id', 'Page Category',
       'Page Admin Top Country', 'Page Description', 'Page Created',
       'Likes at Posting', 'Followers at Posting', 'Post Created',
       'Post Created Date', 'Post Created Time', 'Type', 'Total Interactions',
       'Likes', 'Comments', 'Shares', 'Love', 'Wow', 'Haha', 'Sad', 'Angry',
       'Care', 'Video Share Status', 'Is Video Owner?', 'Post Views',
       'Total Views', 'Total Views For All Crossposts', 'Video Length', 'URL',
       'Message', 'Link', 'Final Link', 'Image Text', 'Link Text',
       'Description', 'Sponsor Id', 'Sponsor Name', 'Sponsor Category',
       'Overperforming Score (weighted  —  Likes 1x Shares 1x Comments 1x Love 1x Wow 1x Haha 1x Sad 1x Angry 1x Care 1x )'],
      dtype='object')

In [6]:
facebook_df["text"] = facebook_df["Message"]

# Обрабатываем случаи, когда Link Text или Description могут быть пустыми
facebook_df["link_text"] = (
    facebook_df["Link Text"].fillna('') + " " + facebook_df["Description"].fillna('')
).str.strip()

# Функция для удаления текста, если он содержит только одно слово
def remove_single_word(text):
    if len(text.split()) <= 1:
        return ""
    return text

# Применяем проверку к колонкам text и link_text
facebook_df["text"] = facebook_df["text"].fillna('').apply(remove_single_word)
facebook_df["link_text"] = facebook_df["link_text"].fillna('').apply(remove_single_word)

facebook_df["text"] = facebook_df["text"].replace("", None).drop_duplicates().fillna("")
facebook_df["link_text"] = facebook_df["link_text"].replace("", None).drop_duplicates().fillna("")

In [7]:
facebook_df.sample(1)


,Group Name,User Name,Facebook Id,Page Category,Page Admin Top Country,Page Description,Page Created,Likes at Posting,Followers at Posting,Post Created,...,Final Link,Image Text,Link Text,Description,Sponsor Id,Sponsor Name,Sponsor Category,Overperforming Score (weighted — Likes 1x Shares 1x Comments 1x Love 1x Wow 1x Haha 1x Sad 1x Angry 1x Care 1x ),text,link_text
6230,Health Care Assistants HCA / Nurses / Support Workers - JOBS (UK),NaN,629521900975276,none,NaN,NaN,NaN,NaN,NaN,2022-03-29 06:45:44 EDT,...,NaN,NaN,This is a re-share of a post,"Chief Nurse to retire after 44 years of loyal NHS service \n\nA senior Warrington nurse will retire this week after more than 44 years of loyal service in the NHS.\n\nMichelle Creed will step down from her role as Chief Nurse of NHS Halton and NHS Warrington Clinical Commissioning Groups (CCG) this Thursday, March 31.\n\nClick the link to read more:",NaN,NaN,NaN,0.74,I thought this was such a lovely post I couldn't not share 😊,"This is a re-share of a post Chief Nurse to retire after 44 years of loyal NHS service \n\nA senior Warrington nurse will retire this week after more than 44 years of loyal service in the NHS.\n\nMichelle Creed will step down from her role as Chief Nurse of NHS Halton and NHS Warrington Clinical Commissioning Groups (CCG) this Thursday, March 31.\n\nClick the link to read more:"


In [8]:
facebook_df.to_excel(DATA_PATH / 'facebook_data_to_model.xlsx', index=False)